# <span style="font-width:bold; font-size: 3rem; color:#1EB182;">**Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Backfill Features to the Feature Store</span>


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/nyc_taxi_fares/1_backfill_feature_groups.ipynb)

## 🗒️ This notebook is divided into 3 sections:
1. **Load some data from files**,
2. **Connect to the Hopsworks Feature Store**,
3. **Create feature groups and insert them into the Feature Store**.

![tutorial-flow](../../images/01_featuregroups.png)

### <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import pandas as pd

___

## <span style='color:#ff5f27'> 💽 Loading Historical Data</span>


### <span style='color:#ff5f27'> 🚖 Rides Data

In [ ]:
df_rides = pd.read_csv("https://repo.hops.works/dev/davit/nyc_taxi/rides500.csv", index_col=0)

df_rides.head()

### <span style='color:#ff5f27'> 💸 Fares Data

In [ ]:
df_fares = pd.read_csv("https://repo.hops.works/dev/davit/nyc_taxi/fares500.csv", index_col=0)

df_fares.head()

---

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

In [ ]:
project

___

## <span style="color:#ff5f27;">🪄 Creating Feature Groups</span>

### <span style='color:#ff5f27'> 🚖 Rides Data

In [ ]:
rides_fg = fs.get_or_create_feature_group(name="nyc_taxi_rides",
                                          version=1,
                                          primary_key=["ride_id"],
                                          event_time=["pickup_datetime"],
                                          description="Rides features",
                                          time_travel_format="HUDI",     
                                          online_enabled=True,                                                
                                          statistics_config=True)
rides_fg.insert(df_rides)

### <span style='color:#ff5f27'> 💸 Fares Data

In [ ]:
fares_fg = fs.get_or_create_feature_group(name="nyc_taxi_fares",
                                          version=1,
                                          primary_key=["ride_id"], 
                                          description="Taxi fares features",
                                          time_travel_format="HUDI",  
                                          online_enabled=True,
                                          statistics_config=True)   
fares_fg.insert(df_fares)

---

## <span style="color:#ff5f27;">⏭️ **Next:** Part 02 </span>

In the next notebook, you will be generating new data for the Feature Groups.

---